In [21]:
import spacy_transformers
import spacy
from spacy.lang.en.examples import sentences 
import torchtext
import pickle
import torch
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import nltk
from nltk.translate.bleu_score import corpus_bleu
from collections import Counter
from torchtext.legacy.data import Field

ModuleNotFoundError: No module named 'torchtext.legacy'

In [22]:
import torchtext
print(torchtext.__version__)

0.16.1+cpu


In [4]:
import json
nlp_en = spacy.load("en_core_web_md")
nlp_de = spacy.load("de_core_news_md")

with open('train.json') as f:
   train_data = json.load(f)

KeyboardInterrupt: 

Die ganze Stadt ist ein Startup: Shenzhen ist das Silicon Valley für Hardware-Firmen
Die DET nk
ganze ADJ nk
Stadt NOUN sb
ist AUX ROOT
ein DET nk
Startup NOUN pd
: PUNCT punct
Shenzhen PROPN sb
ist AUX cj
das DET nk
Silicon PROPN pnc
Valley PROPN sb
für ADP mnr
Hardware-Firmen NOUN nk


In [4]:
with open('validation.json') as f:
   val_data = json.load(f)
with open('test.json') as f:
   test_data = json.load(f)

In [27]:
#first for train data only
data=train_data['data']


In [2]:
train_en_tokenize_file_name='english_tokenized.pkl'
train_de_tokenize_file_name='german_tokenized.pkl'
val_en_tokenize_file_name='val_english_tokenized.pkl'
val_de_tokenize_file_name='val_german_tokenized.pkl'
test_en_tokenize_file_name='test_english_tokenized.pkl'
test_de_tokenize_file_name='test_german_tokenized.pkl'

In [28]:
len(english_sentences)

4522998

In [5]:
def batch_tokenize_input(input_texts, nlp_eng):
    # Tokenize the input texts in batches
    input_tokenized = [nlp_eng(text) for text in input_texts]
    return [[token.text for token in doc] for doc in input_tokenized]

In [7]:
#first Time
def tokenize(data,en_file_name,de_file_name):
    german_sentences = [item['de'] for item in data]
    english_sentences = [item['en'] for item in data]
    # Batch size for tokenization
    batch_size = 1000


    # Tokenize and save in batches
    for i in range(0, len(english_sentences), batch_size):
        batch_english = batch_tokenize_input(english_sentences[i:i+batch_size], nlp_en)
        batch_german = batch_tokenize_input(german_sentences[i:i+batch_size], nlp_de)
        print(batch_size,i)
        # Append tokenized batches to the pickle file
        with open(en_file_name, 'ab') as f:
            pickle.dump(batch_english, f)

        with open(de_file_name, 'ab') as f:
            pickle.dump(batch_german, f)

In [9]:
tokenize(test_data['data'],test_en_tokenize_file_name,test_de_tokenize_file_name)
tokenize(val_data['data'],val_en_tokenize_file_name,val_de_tokenize_file_name)
tokenize(train_data['data'],train_en_tokenize_file_name,train_de_tokenize_file_name)

1000 0
1000 1000
1000 2000
1000 0
1000 1000
1000 2000
1000 3000
1000 0
1000 1000
1000 2000
1000 3000
1000 4000
1000 5000
1000 6000
1000 7000
1000 8000
1000 9000
1000 10000
1000 11000
1000 12000
1000 13000
1000 14000
1000 15000
1000 16000
1000 17000
1000 18000
1000 19000
1000 20000
1000 21000
1000 22000
1000 23000
1000 24000
1000 25000
1000 26000
1000 27000
1000 28000
1000 29000
1000 30000
1000 31000
1000 32000
1000 33000
1000 34000
1000 35000
1000 36000
1000 37000
1000 38000
1000 39000
1000 40000
1000 41000
1000 42000
1000 43000
1000 44000
1000 45000
1000 46000
1000 47000
1000 48000
1000 49000
1000 50000
1000 51000
1000 52000
1000 53000
1000 54000
1000 55000
1000 56000
1000 57000
1000 58000
1000 59000
1000 60000
1000 61000
1000 62000
1000 63000
1000 64000
1000 65000
1000 66000
1000 67000
1000 68000
1000 69000
1000 70000
1000 71000
1000 72000
1000 73000
1000 74000
1000 75000
1000 76000
1000 77000
1000 78000
1000 79000
1000 80000
1000 81000
1000 82000
1000 83000
1000 84000
1000 85000
100

KeyboardInterrupt: 

In [15]:

def build_vocab(tokenized_sentences, min_freq=1):
    # Count the frequency of each word in the dataset
    word_freq = Counter(word for sentence in tokenized_sentences for word in sentence)

    # Filter words by minimum frequency
    vocab = [word for word, freq in word_freq.items() if freq >= min_freq]

    # Create word to index mapping
    word_to_index = {word: idx + 1 for idx, word in enumerate(vocab)}  # +1 for zero padding

    return word_to_index

In [20]:

with open('english_tokenized.pkl', 'rb') as f:
        english_tokenized = pickle.load(f)

    # Load German tokenized sentences from the pickle file
with open('german_tokenized.pkl', 'rb') as f:
        german_tokenized = pickle.load(f)

with open('val_english_tokenized.pkl', 'rb') as f:
        val_english_tokenized = pickle.load(f)

    # Load German tokenized sentences from the pickle file
with open('val_german_tokenized.pkl', 'rb') as f:
        val_german_tokenized = pickle.load(f)

with open('test_english_tokenized.pkl', 'rb') as f:
        train_english_tokenized = pickle.load(f)

    # Load German tokenized sentences from the pickle file
with open('test_german_tokenized.pkl', 'rb') as f:
        train_german_tokenized = pickle.load(f)

all_english = english_tokenized + val_english_tokenized + train_english_tokenized
all_german = german_tokenized + val_german_tokenized + train_german_tokenized
# Create vocabulary
eng_vocab = torchtext.vocab.build_vocab_from_iterator(all_english)
ger_vocab = torchtext.vocab.build_vocab_from_iterator(all_german)
# Save vocabularies
with open('english_vocab.pkl', 'wb') as f:
    pickle.dump(eng_vocab, f)

with open('german_vocab.pkl', 'wb') as f:
    pickle.dump(ger_vocab, f)
len(eng_vocab),len(ger_vocab)


(9507, 12945)

In [38]:
with open('english_vocab.pkl', 'rb') as f:
    eng_vocab = pickle.load(f)

# Load German tokenized sentences from the pickle file
with open('german_vocab.pkl', 'rb') as f:
    ger_vocab = pickle.load(f)

if '<unk>' not in eng_vocab:
    eng_vocab.insert_token('<unk>', 0)  # Adjust the index if needed

if '<unk>' not in ger_vocab:
    ger_vocab.insert_token('<unk>', 0)  # Adjust the index if needed

In [39]:
def preProcess( english_tokenized_file_name,german_tokenized_file_name,eng_vocab,ger_vocab):

    with open(english_tokenized_file_name, 'rb') as f:
        english_tokenized = pickle.load(f)

    # Load German tokenized sentences from the pickle file
    with open(german_tokenized_file_name, 'rb') as f:
        german_tokenized = pickle.load(f)
    # Convert words to indices
    english_indices = [torch.tensor([eng_vocab[word] if word in eng_vocab else eng_vocab['<unk>'] for word in sentence], dtype=torch.int) for sentence in english_tokenized]
    german_indices = [torch.tensor([ger_vocab[word] if word in ger_vocab else eng_vocab['<unk>'] for word in sentence], dtype=torch.int) for sentence in german_tokenized]

    # Pad sequences to the same length
    max_len = max(max(len(seq) for seq in english_indices), max(len(seq) for seq in german_indices))
    english_padded = pad_sequence([torch.cat([seq, torch.zeros(max_len - len(seq))], dim=0) for seq in english_indices], batch_first=True)
    german_padded = pad_sequence([torch.cat([seq, torch.zeros(max_len - len(seq))], dim=0) for seq in german_indices], batch_first=True)
    return english_padded,german_padded, max_len


In [40]:
train_english_padded,train_german_padded,max_len = preProcess(train_en_tokenize_file_name,train_de_tokenize_file_name,eng_vocab,ger_vocab)
#first time
torch.save(train_english_padded, 'english_padded.pt')
torch.save(train_german_padded, 'german_padded.pt')

In [41]:
val_english_padded,val_german_padded,max_len = preProcess(val_en_tokenize_file_name,train_de_tokenize_file_name,eng_vocab,ger_vocab)
#first time
torch.save(train_english_padded, 'english_padded_val.pt')
torch.save(train_german_padded, 'german_padded_val.pt')

In [42]:
# Move data and model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_english_padded = train_english_padded.to(device)
train_german_padded = train_german_padded.to(device)
val_english_padded = val_english_padded.to(device)
val_german_padded = val_german_padded.to(device)

In [43]:
device

device(type='cuda')

In [44]:
# Define the dataset
class TranslationDataset(Dataset):
    def __init__(self, english_data, german_data):
        self.english_data = english_data
        self.german_data = german_data

    def __len__(self):
        return len(self.english_data)

    def __getitem__(self, idx):
        return self.english_data[idx], self.german_data[idx]



In [45]:

# Create DataLoader
train_dataset = TranslationDataset(train_english_padded, train_german_padded)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Create DataLoader
val_dataset = TranslationDataset(val_english_padded, val_german_padded)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=True)

In [46]:

class EncoderLSTM(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout):
        super(EncoderLSTM, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout)

    def forward(self, x):
        # x shape: (seq_length, N)

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (seq_length, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding)
        return hidden, cell

class DecoderLSTM(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout):
        super(DecoderLSTM, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden_size = hidden_size
        self.output_size=output_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        # x shape: (N) but we want (1, N)
        x = x.unsqueeze(0)

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (1, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
        # outputs shape: (1, N, hidden_size)

        predictions = self.fc(outputs)
        # predictions shape: (1, N, length_of_vocab)

        predictions = predictions.squeeze(0)
        return predictions, hidden, cell


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_forcing_ratio = 0.5):
        # source shape: (src_len, N)
        # target shape: (trg_len, N)

        trg_len, N = target.shape
        trg_vocab_size = self.decoder.output_size

        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, N, trg_vocab_size).to(device)

        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(source)

        # first input to the decoder is the <sos> tokens
        x = target[0]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(x, hidden, cell)

            outputs[t] = output

            # decide if we will use teacher forcing or not
            best_guess = output.argmax(1)
            x = target[t] if random.random() < teacher_forcing_ratio else best_guess
        
        return outputs


In [47]:
# Initialize models
INPUT_DIM = len(eng_vocab)  # Assuming eng_vocab is your English vocabulary
OUTPUT_DIM = len(ger_vocab)  # Assuming ger_vocab is your German vocabulary
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
learning_rate=0.001

In [48]:

enc = EncoderLSTM(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT).to(device)
dec = DecoderLSTM(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, OUTPUT_DIM, N_LAYERS, DEC_DROPOUT).to(device)

model = Seq2Seq(enc, dec).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=-1) 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [49]:
# Training loop
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    
    epoch_loss = 0

    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()
        
        output = model(src, trg)
        
        # trg = [trg len, batch size]
        # output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [50]:
def evaluate(model, iterator, criterion):
    model.eval()
    
    epoch_loss = 0

    with torch.no_grad():
        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0)  # turn off teacher forcing

            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [51]:
# Training settings
N_EPOCHS = 10
CLIP = 1
num_epochs = 50
for epoch in range(N_EPOCHS):
    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_dataloader, criterion)

    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Val. Loss: {valid_loss:.3f}')


# Save the model
torch.save(model.state_dict(), 'eng_to_ger_translation_model.pth')

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)

In [16]:
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

0
Tesla V100-PCIE-32GB


In [ ]:
# Inference
def translate(input_sentence):
    input_tokens = tokenize(input_sentence)
    input_indices = torch.tensor([input_vocab[token] for token in input_tokens], dtype=torch.long).unsqueeze(1).to(device)

    encoder_output, encoder_hidden = model.encoder(input_indices)

    # Initialize decoder input with a start token
    decoder_input = torch.tensor([output_vocab['<start>']], dtype=torch.long).to(device)
    decoded_tokens = []

    for _ in range(max_length):
        decoder_output, decoder_hidden = model.decoder(decoder_input, encoder_hidden)
        _, top_index = decoder_output.topk(1)
        decoder_input = top_index.squeeze().detach()
        decoded_tokens.append(list(output_vocab.keys())[list(output_vocab.values()).index(int(decoder_input))])

       



